In [1]:
# Copyright 2020 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_merlin_getting-started-movielens-04-triton-inference-with-tf/nvidia_logo.png" style="width: 90px; float: right;">

## Serve Recommendations from the TensorFlow Model

This notebook is created using the latest stable [merlin-tensorflow](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-tensorflow/tags) container.

The last step is to deploy the ETL workflow and saved model to production. In the production setting, we want to transform the input data as done during training ETL. We need to ensure that our data is processed in the same fashion during training as in production. Therefore, we deploy the NVTabular workflow with the Merlin Model as an ensemble model to Triton Inference Server. The ensemble model guarantees that the same transformations are applied to the raw inputs.

<img src="./imgs/triton-tf.png" width="25%">


### Launching and Starting the Triton Server

Before we get started, you should start the container for Triton Inference Server with the following command. This command includes the `-v` argument that mounts your local `model-repository` folder with your saved models from the previous notebook (`03a-Training-with-TF.ipynb`) to the `/model` directory in the container.

```
docker run -it --gpus device=0 -p 8000:8000 -p 8001:8001 -p 8002:8002 -v ${PWD}:/model/ nvcr.io/nvidia/merlin/merlin-tensorflow:latest
```


After you started the container, you can start Triton Inference Server with the following command.
You need to provide correct path for the `models` directory.

```
tritonserver --model-repository=path_to_models --backend-config=tensorflow,version=2 --model-control-mode=explicit 
```

Note: The model-repository path is `/root/nvt-examples/models`. The models haven't been loaded, yet. Below, we will request the Triton server to load the saved ensemble model.

In [2]:
# External dependencies
import os
from time import time

# Get dataframe library - cudf or pandas
from merlin.core.dispatch import get_lib
df_lib = get_lib()

import tritonclient.grpc as grpcclient
import nvtabular.inference.triton as nvt_triton

We define our base directory, containing the data.

In [3]:
# path to preprocessed data
INPUT_DATA_DIR = os.environ.get(
    "INPUT_DATA_DIR", os.path.expanduser("~/nvt-examples/movielens/data/")
)

Let's deactivate the warnings before sending requests.

In [4]:
import warnings

warnings.filterwarnings("ignore")

### Loading Ensemble Model with Triton Inference Serve

At this stage, you should have launched the Triton Inference Server docker container with the instructions above.

Let's connect to the Triton Inference Server. Use Triton’s ready endpoint to verify that the server and the models are ready for inference. Replace localhost with your host ip address.

In [5]:
import tritonhttpclient

try:
    triton_client = tritonhttpclient.InferenceServerClient(url="localhost:8000", verbose=True)
    print("client created.")
except Exception as e:
    print("channel creation failed: " + str(e))

client created.


/usr/local/lib/python3.8/dist-packages/tritonhttpclient/__init__.py:31: DeprecationWarning: The package `tritonhttpclient` is deprecated and will be removed in a future version. Please use instead `tritonclient.http`
  warnings.warn(


In [6]:
import warnings

warnings.filterwarnings("ignore")

We check if the server is alive.

In [8]:
triton_client.is_server_live()

GET /v2/health/live, headers None
<HTTPSocketPoolResponse status=200 headers={'content-length': '0', 'content-type': 'text/plain'}>


True

The HTTP request returns status 200 if Triton is ready and non-200 if it is not ready.

We check the available models in the repositories:

In [12]:
triton_client.get_model_repository_index()

POST /v2/repository/index, headers None

<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '21'}>
bytearray(b'[{"name":"ensemble"}]')


[{'name': 'ensemble'}]

We load the ensemble model.

In [13]:
%%time

triton_client.load_model(model_name="ensemble")

POST /v2/repository/models/ensemble/load, headers None
{}
<HTTPSocketPoolResponse status=400 headers={'content-type': 'application/json', 'content-length': '75'}>


InferenceServerException: failed to load 'ensemble', failed to poll from model repository

### Send a Request to Triton Inference Server to Transform a Raw Dataset

A minimal model repository for a TensorFlow SavedModel model is:
```
  <model-repository-path>/<model-name>/
      config.pbtxt
      1/
        model.savedmodel/
           <saved-model files>
```
Let's check out our model repository layout. You can install tree library with apt-get install tree, and then run `!tree /model/models/` to print out the model repository layout as below:
               
```
/model/models/
|-- movielens
|   |-- 1
|   `-- config.pbtxt
|-- movielens_nvt
|   |-- 1
|   |   |-- __pycache__
|   |   |   `-- model.cpython-38.pyc
|   |   |-- model.py
|   |   `-- workflow
|   |       |-- categories
|   |       |   |-- unique.genres.parquet
|   |       |   |-- unique.movieId.parquet
|   |       |   `-- unique.userId.parquet
|   |       |-- metadata.json
|   |       `-- workflow.pkl
|   `-- config.pbtxt
`-- movielens_tf
    |-- 1
    |   `-- model.savedmodel
    |       |-- assets
    |       |-- saved_model.pb
    |       `-- variables
    |           |-- variables.data-00000-of-00001
    |           `-- variables.index
    `-- config.pbtxt
```
You can see that we have a `config.pbtxt` file. Each model in a model repository must include a model configuration that provides required and optional information about the model. Typically, this configuration is provided in a `config.pbtxt` file specified as [ModelConfig protobuf](https://github.com/triton-inference-server/server/blob/r20.12/src/core/model_config.proto).

Let's read the raw validation set, and send 3 rows of `userId` and `movieId` as input to the saved NVTabular model.

In [53]:
# read in the workflow (to get input/output schema to call triton with)
batch = df_lib.read_parquet(
    os.path.join(INPUT_DATA_DIR, "valid.parquet"), num_rows=3, columns=["userId", "movieId"]
)
print(batch)

          userId  movieId
5867328    37999     8874
13455621   87008     2710
15095829   97836     2797


In [54]:
inputs = nvt_triton.convert_df_to_triton_input(["userId", "movieId"], batch, grpcclient.InferInput)

outputs = [
    grpcclient.InferRequestedOutput(col)
    for col in ["userId", "movieId"]
]

MODEL_NAME_NVT = os.environ.get("MODEL_NAME_NVT", "movielens_nvt")

with grpcclient.InferenceServerClient("localhost:8001") as client:
    response = client.infer(MODEL_NAME_NVT, inputs, request_id="1", outputs=outputs)

for col in ["userId", "movieId"]:
    print(col, response.as_numpy(col), response.as_numpy(col).shape)

userId [[37999]
 [87008]
 [97836]] (3, 1)
movieId [[8874]
 [2710]
 [2797]] (3, 1)


You might notice that we don't need to send the genres column as an input. The reason for that is the nvt model will look up the genres for each movie as part of the `JoinExternal` op it applies. Also notice that when creating the request for the `movielens_nvt` model, we return 2 columns (values and nnzs) for the `genres` column rather than 1.

### Build the End-To-End Inference Pipeline

We will do the same, but this time we directly read in first 3 rows of the the raw `valid.parquet` file with cuDF.

In [59]:
response.get_response()

model_name: "movielens_nvt"
model_version: "1"
id: "1"
outputs {
  name: "userId"
  datatype: "INT64"
  shape: 3
  shape: 1
}
outputs {
  name: "movieId"
  datatype: "INT64"
  shape: 3
  shape: 1
}
raw_output_contents: "o\224\000\000\000\000\000\000\340S\001\000\000\000\000\000,~\001\000\000\000\000\000"
raw_output_contents: "\252\"\000\000\000\000\000\000\226\n\000\000\000\000\000\000\355\n\000\000\000\000\000\000"

In [70]:
triton_client.load_model(model_name="movielens_tf")
triton_client.load_model(model_name="movielens")

# read in the workflow (to get input/output schema to call triton with)
batch = df_lib.read_parquet(
    os.path.join(INPUT_DATA_DIR, "valid.parquet"), num_rows=3, columns=["userId", "movieId"]
)

print("raw data:\n", batch, "\n")

# convert the batch to a triton inputs
inputs = nvt_triton.convert_df_to_triton_input(["userId", "movieId"], batch, grpcclient.InferInput)

# placeholder variables for the output
outputs = [grpcclient.InferRequestedOutput("rating")]

MODEL_NAME_ENSEMBLE = os.environ.get("MODEL_NAME_ENSEMBLE", "movielens")

# build a client to connect to our server.
# This InferenceServerClient object is what we'll be using to talk to Triton.
# make the request with tritonclient.grpc.InferInput object

with grpcclient.InferenceServerClient("localhost:8001") as client:
    response = client.infer(MODEL_NAME_ENSEMBLE, inputs, request_id="1", outputs=outputs)

POST /v2/repository/models/movielens_tf/load, headers None
{}
<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '0'}>
Loaded model 'movielens_tf'
POST /v2/repository/models/movielens/load, headers None
{}
<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '0'}>
Loaded model 'movielens'
raw data:
           userId  movieId
5867328    37999     8874
13455621   87008     2710
15095829   97836     2797 



In [71]:
response.get_response()

model_name: "movielens"
model_version: "1"
id: "1"
parameters {
  key: "sequence_end"
  value {
    bool_param: false
  }
}
parameters {
  key: "sequence_id"
  value {
    int64_param: 0
  }
}
parameters {
  key: "sequence_start"
  value {
    bool_param: false
  }
}
outputs {
  name: "rating"
  datatype: "FP32"
  shape: 3
  shape: 1
}
raw_output_contents: "1\014 ?\250!#?^p!?"

In [67]:
print("predicted sigmoid result:\n", response.as_numpy("outputs"))

predicted sigmoid result:
 None


Let's send request for a larger batch size and measure the total run time and throughput.

In [13]:
# read in the workflow (to get input/output schema to call triton with)
batch_size = 64
batch = df_lib.read_parquet(
    os.path.join(INPUT_DATA_DIR, "valid.parquet"),
    num_rows=batch_size,
    columns=["userId", "movieId"],
)

start = time()
# convert the batch to a triton inputs
inputs = nvt_triton.convert_df_to_triton_input(["userId", "movieId"], batch, grpcclient.InferInput)

# placeholder variables for the output
outputs = [grpcclient.InferRequestedOutput("output")]

MODEL_NAME_ENSEMBLE = os.environ.get("MODEL_NAME_ENSEMBLE", "movielens")

# build a client to connect to our server.
# This InferenceServerClient object is what we'll be using to talk to Triton.
# make the request with tritonclient.grpc.InferInput object

with grpcclient.InferenceServerClient("localhost:8001") as client:
    response = client.infer(MODEL_NAME_ENSEMBLE, inputs, request_id="1", outputs=outputs)

t_final = time() - start
print("predicted sigmoid result:\n", response.as_numpy("output"), "\n")

print(f"run_time(sec): {t_final} - rows: {batch_size} - inference_thru: {batch_size / t_final}")

predicted sigmoid result:
 [[0.6152474 ]
 [0.6197106 ]
 [0.6189732 ]
 [0.61421186]
 [0.61287224]
 [0.612817  ]
 [0.61092085]
 [0.6234857 ]
 [0.61349696]
 [0.61103773]
 [0.61636466]
 [0.61942095]
 [0.6194856 ]
 [0.6162945 ]
 [0.60351753]
 [0.609695  ]
 [0.62262255]
 [0.6180354 ]
 [0.6175308 ]
 [0.61908466]
 [0.6161449 ]
 [0.6128701 ]
 [0.60643166]
 [0.616207  ]
 [0.616828  ]
 [0.6228604 ]
 [0.6162526 ]
 [0.6177362 ]
 [0.6238064 ]
 [0.6199255 ]
 [0.6200201 ]
 [0.61577153]
 [0.6227766 ]
 [0.6123207 ]
 [0.6177533 ]
 [0.6164962 ]
 [0.6176095 ]
 [0.6182202 ]
 [0.6162337 ]
 [0.615147  ]
 [0.61689186]
 [0.6206259 ]
 [0.6219693 ]
 [0.6189993 ]
 [0.60714984]
 [0.61736315]
 [0.6237706 ]
 [0.6075932 ]
 [0.6135807 ]
 [0.6114176 ]
 [0.6124701 ]
 [0.61637044]
 [0.6148614 ]
 [0.62120986]
 [0.60603315]
 [0.61048526]
 [0.61690056]
 [0.61266035]
 [0.6128772 ]
 [0.6178152 ]
 [0.61760587]
 [0.61757046]
 [0.6055966 ]
 [0.6156989 ]] 

run_time(sec): 0.01111745834350586 - rows: 64 - inference_thru: 5756.71147

Let's unload all the models.

In [14]:
triton_client.unload_model(model_name="movielens")
triton_client.unload_model(model_name="movielens_nvt")
triton_client.unload_model(model_name="movielens_tf")

POST /v2/repository/models/movielens/unload, headers None
{"parameters":{"unload_dependents":false}}
<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '0'}>
Loaded model 'movielens'
POST /v2/repository/models/movielens_nvt/unload, headers None
{"parameters":{"unload_dependents":false}}
<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '0'}>
Loaded model 'movielens_nvt'
POST /v2/repository/models/movielens_tf/unload, headers None
{"parameters":{"unload_dependents":false}}
<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '0'}>
Loaded model 'movielens_tf'
